In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser

In [2]:
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

## NASA Mars News

In [3]:
nasa_mars_url = "https://mars.nasa.gov/news/"

browser = init_browser()
browser.visit(nasa_mars_url)
soup = BeautifulSoup(browser.html, 'html.parser')

In [4]:
news_title = soup.find_all("div", class_="content_title")[0].get_text()
news_title

"NASA's Curiosity Rover Finds an Ancient Oasis on Mars"

In [5]:
news_paragraph = soup.find_all("div", class_="article_teaser_body")[0].get_text()
news_paragraph

"New evidence suggests salty, shallow ponds once dotted a Martian crater — a sign of the planet's drying climate."

In [6]:
browser.quit()

## JPL Mars Space Images - Featured Image

In [7]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

browser = init_browser()
browser.visit(jpl_url)

In [8]:
browser.click_link_by_partial_text(news_title)

In [9]:
browser.click_link_by_partial_text("Full image and caption")

In [10]:
browser.click_link_by_partial_text(".jpg")

In [11]:
featured_image_url = browser.url
featured_image_url

'https://photojournal.jpl.nasa.gov/jpeg/PIA21261.jpg'

In [12]:
browser.quit()

## Mars Weather

In [13]:
twitter_url = "https://twitter.com/marswxreport?lang=en"
twitter_page = requests.get(twitter_url)

soup = BeautifulSoup(twitter_page.content, 'html.parser')

In [14]:
first_tweet = soup.find_all("div", class_="js-tweet-text-container")[0]
mars_weather_tweet = first_tweet.find("p").get_text()
print(mars_weather_tweet)

InSight sol 310 (2019-10-10) low -102.2ºC (-152.0ºF) high -26.6ºC (-15.8ºF)
winds from the SSE at 5.0 m/s (11.2 mph) gusting to 19.1 m/s (42.8 mph)
pressure at 7.20 hPapic.twitter.com/0Pt27sbe1R


## Mars Facts

In [15]:
mars_facts_url = "https://space-facts.com/mars/"

browser = init_browser()
browser.visit(mars_facts_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [16]:
mars_facts_table = soup.find_all("table", id="tablepress-p-mars")[0]

In [17]:
# get the types of facts
topics = []
for row in mars_facts_table.find_all("td",class_="column-1"):
    topic_text = row.get_text().replace(":","")
    topics.append(topic_text)
topics

['Equatorial Diameter',
 'Polar Diameter',
 'Mass',
 'Moons',
 'Orbit Distance',
 'Orbit Period',
 'Surface Temperature ',
 'First Record',
 'Recorded By']

In [18]:
# get the measurements of the facts
measurements = []
for row in mars_facts_table.find_all("td",class_="column-2"):
    measurements.append(row.get_text())
measurements

['6,792 km',
 '6,752 km',
 '6.39 × 10^23 kg (0.11 Earths)',
 '2 (Phobos & Deimos)',
 '227,943,824 km (1.38 AU)',
 '687 days (1.9 years)',
 '-87 to -5 °C',
 '2nd millennium BC',
 'Egyptian astronomers']

In [19]:
# combine topics and measurements into single dictionary
mars_facts_dict = {}

for i in range(len(topics)):
    mars_facts_dict[topics[i]] = measurements[i]
mars_facts_dict

{'Equatorial Diameter': '6,792 km',
 'Polar Diameter': '6,752 km',
 'Mass': '6.39 × 10^23 kg (0.11 Earths)',
 'Moons': '2 (Phobos & Deimos)',
 'Orbit Distance': '227,943,824 km (1.38 AU)',
 'Orbit Period': '687 days (1.9 years)',
 'Surface Temperature ': '-87 to -5 °C',
 'First Record': '2nd millennium BC',
 'Recorded By': 'Egyptian astronomers'}

In [20]:
browser.quit()

## Mars Hemispheres

In [21]:
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

browser = init_browser()
browser.visit(usgs_url)

In [22]:
hemispheres = ["Cerberus","Schiaparelli","Syrtis","Valles"]
hemisphere_image_urls = []

# loop through each hemisphere and scrape the data
for hemi in hemispheres:
    new_dict = {}

    browser.click_link_by_partial_text(hemi)
    usgs_html = browser.html
    soup = BeautifulSoup(usgs_html, 'html.parser')
    new_dict["title"] = soup.find("h2").get_text().replace("Enhanced","").strip()
    new_dict["img_url"] = soup.find_all("div", class_="downloads")[0].find_all("a")[0]["href"]
    hemisphere_image_urls.append(new_dict)
    
    # go back to original page with all the hemispheres
    browser.back()

In [23]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [24]:
browser.quit()